In [1]:
"""将100*100的网格再次下采样为50*50的网格,并生成uv坐标"""

'将100*100的网格再次下采样为50*50的网格,并生成uv坐标'

In [67]:
import open3d as o3d
file_path=r'D:\project\image_process\Surface-flatting\doc3D-renderer\src_obj\1_8.obj'
mesh= o3d.io.read_triangle_mesh(file_path)
mesh.compute_vertex_normals()
o3d.visualization.draw_geometries([mesh])


In [68]:
"""查看原始obj信息"""
import numpy as np

mesh_vertices=np.array(mesh.vertices)
print(mesh)
print(19602*3)
print(mesh_vertices.shape)
print(mesh_vertices[899])
#他给的样例网格是有uv坐标的
print(np.asarray(mesh.triangle_uvs).shape)
print(np.asarray(mesh.triangle_uvs)[:10])

TriangleMesh with 10000 points and 19602 triangles.
58806
(10000, 3)
[ 1.029989   -0.74108499  0.22658899]
(58806, 2)
[[1.         0.        ]
 [1.         0.010101  ]
 [0.98989898 0.        ]
 [0.98989898 0.        ]
 [1.         0.010101  ]
 [0.98989898 0.010101  ]
 [1.         0.010101  ]
 [1.         0.020202  ]
 [0.98989898 0.010101  ]
 [0.98989898 0.010101  ]]


In [4]:
x_max,x_min=max(mesh_vertices[:,0]),min(mesh_vertices[:,0])
print(f"x max and min are:{x_max},{x_min}")
y_max,y_min=max(mesh_vertices[:,1]),min(mesh_vertices[:,1])
print(f"y max and min are:{y_max},{y_min}")
z_max,z_min=max(mesh_vertices[:,2]),min(mesh_vertices[:,2])
print(f"z max and min are:{z_max},{z_min}")

x max and min are:1.0597009658813477,-1.0438770055770874
y max and min are:0.8721259832382202,-0.8661559820175171
z max and min are:0.2705099880695343,-0.17642100155353546


In [6]:
#对网格二次采样为50*50大小，50*50是指在水平面上的大小
#沿着x轴均匀采样50个点，均匀即采样点间隔一样，再沿着y轴再采样50，则共2500
#相当于从俯视的角度采样
pcd_u=mesh.sample_points_uniformly(number_of_points=2500)
o3d.visualization.draw_geometries([pcd_u])

In [7]:
distances = pcd_u.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist
mesh_u=o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd_u,o3d.utility.DoubleVector([radius, radius * 2]))
o3d.visualization.draw_geometries([mesh_u])

In [8]:
""""
uv坐标的含义是：
将3d mesh映射到2d纹理图上的坐标，3d mesh上的每一个顶点都有一个uv值，所以他的shape是
3*triangles,2，即每个三角形的每个顶点有的一对二维坐标(u,v)
"""
print(mesh_u)
#采样生成的网格是没有uv坐标的
print(np.asarray(mesh_u.triangle_uvs).shape)

TriangleMesh with 2500 points and 4893 triangles.
(0, 2)


In [9]:
#构造三角形的顶点集合，该集合大于mesh的所有点集合，因为很多点是重复被使用的
#先看官方样例的triangles是不是索引
idx_1=mesh.triangles[0]
idx_2=mesh.triangles[1]
print(idx_1,idx_2)
print(mesh_vertices[idx_1])
print(mesh_vertices[idx_2])

[0 1 2] [2 1 3]
[[-1.01677406 -0.859007    0.265349  ]
 [-1.00517297 -0.85920501  0.257806  ]
 [-1.01674199 -0.848427    0.265775  ]]
[[-1.01674199 -0.848427    0.265775  ]
 [-1.00517297 -0.85920501  0.257806  ]
 [-1.00511503 -0.84866798  0.25859499]]


In [10]:
triangle_vetices_indexs=np.asarray(mesh_u.triangles)
mesh_u_vertices=np.asarray(mesh_u.vertices)
triangle_vetices=np.array([mesh_u_vertices[idx] for idx in triangle_vetices_indexs]).reshape(-1,3)
print(triangle_vetices.shape,4888*3)

(14679, 3) 14664


In [11]:
#插值计算uv坐标
#四个角点
uv_coords=np.array([[0,0],[0,1],[1,0],[1,1]])
#需要计算的插值点数
n_vertices=triangle_vetices.shape[0]
#初始化插值uv坐标
interpolated_uv_coords=np.zeros((n_vertices,2))

#计算所有顶点的x，y范围
x_range=np.max(triangle_vetices[:,0])-np.min(triangle_vetices[:,0])
y_range=np.max(triangle_vetices[:,1])-np.min(triangle_vetices[:,1])
for i in range(n_vertices):

    #计算每个顶点的x，y坐标的归一化值
    x_normalized=(triangle_vetices[i,0]-np.min(triangle_vetices[:,0]))/x_range
    y_normalized=(triangle_vetices[i,1]-np.min(triangle_vetices[:,1]))/y_range

    #根据归一化坐标进行插值
    interpolated_uv_coords[i,0]=uv_coords[0,0]+x_normalized*(uv_coords[2,0]-uv_coords[0,0])
    interpolated_uv_coords[i,1]=uv_coords[0,1]+y_normalized*(uv_coords[1,1]-uv_coords[0,1])

#将插值结果赋给mesh_u
mesh_u.triangle_uvs=o3d.utility.Vector2dVector(interpolated_uv_coords)

In [12]:
print(np.asarray(mesh_u.triangle_uvs).shape)

(14679, 2)


In [2]:
"""网格增强与对齐，以下使用100*100的网格"""

'网格增强与对齐'

In [11]:
"""通过使用网格旋转,拼接的方式生成翻转点云，最后再生成网格"""

'通过使用网格旋转,拼接的方式生成翻转点云，最后再生成网格'

In [14]:
import copy
pcd=mesh.sample_points_uniformly(10000)

In [15]:
import csv
tex_list=r'D:\project\image_process\Surface-flatting\doc3D-renderer\tex.csv'
with open(tex_list,'r') as t:
    texlist=list(csv.reader(t))
    print(texlist)

[['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_1.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_2.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_3.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_4.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_5.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_6.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_7.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_8.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_9.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_10.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_11.jpg'], ['D:\\project\\image_process\\Surface-flatting\\doc3D-renderer\\tex\\r_12.jpg'], ['D:\\project\\image_process\\Surfac

In [18]:
import random
random.randint(0,2)

0

In [24]:
"""以下是使用上述代码构建长方体的100*100网格"""

'以下是使用上述代码构建长方体的100*100网格'

In [8]:
import open3d as o3d
import numpy as np

# 生成100x100的网格坐标
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
xx, yy = np.meshgrid(x, y)
grid_coordinates = np.vstack((xx.ravel(), yy.ravel(), np.zeros_like(xx.ravel()))).T

# 创建点云
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(grid_coordinates)

# 创建三维网格
triangles = []
for i in range(99):
    for j in range(99):
        vertex1 = i * 100 + j
        vertex2 = i * 100 + j + 1
        vertex3 = (i + 1) * 100 + j
        vertex4 = (i + 1) * 100 + j + 1
        triangles.append([vertex1, vertex2, vertex3])
        triangles.append([vertex2, vertex4, vertex3])

mesh = o3d.geometry.TriangleMesh()
mesh.vertices = point_cloud.points
mesh.triangles = o3d.utility.Vector3iVector(triangles)

# 可视化点云和三维网格
o3d.visualization.draw_geometries([mesh])


In [9]:
print(mesh)

TriangleMesh with 10000 points and 19602 triangles.


In [10]:
triangle_vetices_indexs=np.asarray(mesh.triangles)
mesh_vertices=np.asarray(mesh.vertices)
triangle_vetices=np.array([mesh_vertices[idx] for idx in triangle_vetices_indexs]).reshape(-1,3)
print(triangle_vetices.shape,19602*3)

(58806, 3) 58806


In [11]:
#插值计算uv坐标
#四个角点
uv_coords=np.array([[0,0],[0,1],[1,0],[1,1]])
#需要计算的插值点数
n_vertices=triangle_vetices.shape[0]
#初始化插值uv坐标
interpolated_uv_coords=np.zeros((n_vertices,2))

#计算所有顶点的x，y范围
x_range=np.max(triangle_vetices[:,0])-np.min(triangle_vetices[:,0])
y_range=np.max(triangle_vetices[:,1])-np.min(triangle_vetices[:,1])
for i in range(n_vertices):

    #计算每个顶点的x，y坐标的归一化值
    x_normalized=(triangle_vetices[i,0]-np.min(triangle_vetices[:,0]))/x_range
    y_normalized=(triangle_vetices[i,1]-np.min(triangle_vetices[:,1]))/y_range

    #根据归一化坐标进行插值
    interpolated_uv_coords[i,0]=uv_coords[0,0]+x_normalized*(uv_coords[2,0]-uv_coords[0,0])
    interpolated_uv_coords[i,1]=uv_coords[0,1]+y_normalized*(uv_coords[1,1]-uv_coords[0,1])

#将插值结果赋给mesh_u
mesh.triangle_uvs=o3d.utility.Vector2dVector(interpolated_uv_coords)

In [5]:
print(np.asarray(mesh.triangle_uvs).shape)

(58806, 2)


In [12]:
o3d.io.write_triangle_mesh(r"D:\project\image_process\Surface-flatting\doc3D-renderer\obj\rec.obj", mesh)

True

In [17]:
import cv2
hdr_img=cv2.imread(r"D:\project\image_process\Surface-flatting\doc3D-renderer\env\0011.hdr",cv2.IMREAD_ANYDEPTH | cv2.IMREAD_COLOR)
cv2.imshow('aa',hdr_img)
cv2.waitKey()


-1